In [ ]:
from clickhouse_driver import Client
import pandas as pd
import json

client = Client(host='localhost', port=9000, user='default', password='pivanet')

query = 'SELECT parameters, script_id from db_test.logss'
result = client.execute(query)

df = pd.DataFrame(result, columns=['parameters', 'script_id'])

In [99]:
df

,parameters,script_id
0,{},17424b97-bf0a-45f9-9ce2-062eb39f3c37
1,,17424b97-bf0a-45f9-9ce2-062eb39f3c37
2,"{""TYPE_EQUIPMENT"": """"}",17424b97-bf0a-45f9-9ce2-062eb39f3c37
3,,17424b97-bf0a-45f9-9ce2-062eb39f3c37
4,"{""TYPE_EQUIPMENT"": ""оборудованием""}",17424b97-bf0a-45f9-9ce2-062eb39f3c37
...,...,...
755,"{""SERVICE_STATUS_LOCAL"": ""Включена""}",a67dfe4b-a531-4366-9d7c-cfa348256b20
756,,a67dfe4b-a531-4366-9d7c-cfa348256b20
757,"{""SERVICE_ID"": ""160502"", ""ACCOUNT_NUMBER"": ""ac...",a67dfe4b-a531-4366-9d7c-cfa348256b20
758,,a67dfe4b-a531-4366-9d7c-cfa348256b20


In [100]:
def expand_interaction_topics(df, column_name='parameters', id_column='script_id'):
    # Функция для преобразования JSON строки в словарь
    def parse_json(x):
        try:
            return json.loads(x)
        except (json.JSONDecodeError, TypeError):
            return x  # Возвращаем исходное значение, если это уже словарь или если ошибка
    
    # Применяем функцию парсинга
    df[column_name] = df[column_name].apply(parse_json)
    
    df_list = []
    
    for index, row in df.iterrows():
        # Нормализуем вложенные данные из 'INTERACTION_TOPICS'
        if 'INTERACTION_TOPICS' in row[column_name]:
            topics_df = pd.json_normalize(row[column_name], record_path='INTERACTION_TOPICS',
                                          meta=[k for k in row[column_name].keys() if k != 'INTERACTION_TOPICS' and k != id_column] + [id_column],
                                          errors='ignore')
            # Добавляем обработанные данные в список
            df_list.append(topics_df)
        else:
            # Если 'INTERACTION_TOPICS' отсутствует, добавляем строку без изменений
            other_data_df = pd.DataFrame([row[column_name]], index=[index])
            other_data_df[id_column] = row[id_column]  # Добавляем script_id в DataFrame
            df_list.append(other_data_df)
    
    # Конкатенация всех DataFrame в один
    final_df = pd.concat(df_list, ignore_index=True)
    
    return final_df

expanded_df = expand_interaction_topics(df)

C:\Users\serge\AppData\Local\Temp\ipykernel_23068\1530498939.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(df_list, ignore_index=True)


In [101]:
expanded_df

,script_id,0,TYPE_EQUIPMENT,CATALOG,CATALOG_FILTER,$$$taskId,$$$system,BUSINESS_KEY,NO_CACHE,HAS_IN_CATALOG,GET_CATALOG_RECORDS_QUANTITY,CLIENT_ID,ACCOUNT_NUMBER,HARDWARE_AVAILABILITY_AXIROS,CONNECTION_DIAGRAM,THERE_ROUTER,OPERATING_SYSTEM,PROBLEM_RESOLVED,REG_REASON,COMMUNICATION_THEME,COMMUNICATION_DETAIL,COMMUNICATION_RESULT,GUID_THEME,GUID_DETAIL,GUID_RESULT,COMMUNICATION_NUMBER,MARKETING_OFFERS,STATUS_MARKETING_OFFER,REPLACING_CABLE,THERE_IS_A_PHONE_NUMBER_FOR_THE_JOB,GET_CLIENT_TIMEZONE,EMPLOYEE_MRF,theme.title,theme.id,theme.guid,detail.title,detail.id,detail.guid,CLIENT_TIMEZONE,SERVICE_ID,FOR_LAST,FOR_LAST_UNIT,HAS_COMMENT,GET_TASK_QUANTITY,TASK_QUANTITY_NEW,TASK_ARRAY,SEARCH_CASE_STATE,CASE_TYPES,TASK_NUMBER,TASK_DESCRIPTION,result.title,result.id,result.guid,CHECK_NSC,TESTER,GET_CLIENT_PROFILE_ID,GET_SEGMENT,GET_SUBSEGMENT,GET_CONVERGENT_ACCOUNT_FLAG,GET_CONVERGENT_ACCOUNT_NUMBER,PAYMENT_SCHEME,GET_ELK_ID,GET_ELK_LOGIN_BY_ACCOUNT_NUMBER,GET_ACCOUNT_STATUS,EMPLOYEE_TIMEZONE,SSO_ID,SERVICE_ID_2,CRM_DEPARTMENT_INITIATOR,SYSTEM_LOGIN,PRODUCT_ID,ACCOUNT_NUMBER_1,SERVICE_ID_1,PRODUCT_ID_1,ADDENDUM_1,ADDENDUM_2,ADDENDUM_3,ADDENDUM_4,ADDENDUM_5,ROUND_UP,ADDITION,SELECTED_SERVICES_QUANTITY_,SELECTED_PRODUCT,TASK_PRIORITY,TASK_FEEDBACK_TYPE,TASK_FEEDBACK_TIME_FROM,TASK_FEEDBACK_TIME_TO,CRM_DEPARTMENT,CLIENT_ID_AMDOCS,ORGANIZATION_ID,TASK_FEEDBACK_TYPE_VALUE,CALLER_ID,GET_EMAIL,GET_MOBILE_PHONE,GET_CONTACT_PHONE,TERMINATION_SERVICES,AMOUNT_RECALCULATION_MEETS_CRITERIA,TIMEZONE,COMMUNICATION_ARRAY,COMMUNICATION_QUANTITY_NEW,CATALOG_FIELD_VALUE_TEXT,CATALOG_FIELD_VALUE_DATE,CHECKING_POSSIBILITY_CHANGING_CONNECTION,COMPLETE_DIAGNOSIS,CLIENT_EQUIPMENT_BRAND,CLIENT_EQUIPMENT_ID,CLIENT_EQUIPMENT_MODEL,CLIENT_EQUIPMENT_HARDWARE_VERSION,LAST_EQUIPMENT_NETWORK_APPEARANCE_DATE_TIME,MRF,REGION_FILIAL,PORT_HSI_SERVICES,LAN_CONNECTION,OPERATING_SYSTEM_NEW,CLIENT_APPEAL,COMMUNICATION_CHANNEL,COMMUNICATION_DIRECTION,NEW_CLIENT,PROBLEM_NUMBER,CRM_LOGIN,THEME_DETAIL_RESULT,COMMENT,HAS_AVAILABLE_OFFERS,COMMUNICATION_ID,EMPLOYEE_SECOND_NAME,EMPLOYEE_FIRST_NAME,EMPLOYEE_EMAIL,CLIENT_TYPE,CLIENT_SECOND_NAME,CLIENT_FIRST_NAME,CLIENT_MIDDLE_NAME,CLIENT_MRF,SCRIPT_PLAYER_ID,HAS_ACTIVE_CALL,SKILL_GROUP_NAME,SEARCH_REASON,SYSTEM_SOURCE,REGIONAL_OFFICE_ADDRESS,CHOOSING_ANOTHER_REASON,LP_LINK,GET_CLIENT_TYPE,GET_ORGANIZATION_ID,GET_CLIENT_IS_TELE2,GET_CLIENT_IS_DZO,GET_DZO_NAME,CLIENT_DZO,SET_INTERACTION_SERVICE,SELECTED_SERVICES_QUANTITY,PERIOD_START_DATE_5,SERVICE_NAME_2,TASK_CREATE_DATE,VISIT_DATE,PROBLEM_DATE_BEGIN,CURRENT_SCRIPT_SUBOWNER_LOGIN,MAIN_SCRIPT_RUN_ID,PRODUCT_NAME_2,CURRENT_SCRIPT_NAME,TARGET_SCRIPT_NAME,SERVICE_LOGIN_1,PERIOD_END_DATE,PROBLEM_CLOSE_DATE,FLAT_NUMBER_2,SERVICE_LOGIN_2,PRODUCT_ID_2,NUMBER_TRANSFER_ DATE,SERVICE_END_DATE_3,PERIOD_START_DATE_4,SERVICE_START_DATE,HOUSE_ID_1,PERIOD_END_DATE_3,PERIOD_END_DATE_4,VISIT_TIME_FROM,PERIOD_START_DATE_3,INSTALLER_VISIT_TIME_TO,PERIOD_START_DATE_1,SERVICE_STATUS_2,ERROR_INTEGRATION_DATETIME,FLAT_NUMBER_1,LINE_NUMBER_1,PERIOD_END_DATE_1,SERVICE_START_DATE_3,SERVICE_END_DATE_4,MAIN_SCRIPT_OWNER_LOGIN,CURRENT_SCRIPT_VERSION,BIRTH_DATE,PERIOD_END_DATE_2,SERVICE_STATUS_1,ID_CARD_END_DATE,INSTALLER_VISIT_DATE,INSTALLER_VISIT_TIME_FROM,LINE_NUMBER_2,MAIN_SCRIPT_ID,PERIOD_START_DATE,CURRENT_SCRIPT_ID,SERVICE_NAME_1,SERVICE_END_DATE,SERVICE_START_DATE_4,VISIT_TIME_TO,TASK_CLOSE_DATE,SERVICE_END_DATE_2,SERVICE_END_DATE_5,ACCOUNT_NUMBER_2,PERIOD_START_DATE_2,DESIRABLE_TERMINATION_DATE,SELECTED_SERVICES_LIST,SERVICE_ADDRESS_1,SERVICE_START_DATE_5,SERVICE_END_DATE_1,SERVICE_START_DATE_2,SELECTED_PRODUCTS_LIST,SERVICE_START_DATE_1,PERIOD_END_DATE_5,HOUSE_ID_2,PRODUCT_NAME_1,ID_CARD_ISSUE_DATE,SERVICE_ADDRESS_2,DESIRED_TERMINATION_DATE,CURRENT_SCRIPT_RUN_ID,MAIN_SCRIPT_SUBOWNER_LOGIN,CURRENT_SCRIPT_OWNER_LOGIN,MAIN_SCRIPT_VERSION,MAIN_SCRIPT_NAME,PROBLEM_EXPECTED_CLOSE_DATE,GET_FILTERED_SERVICES_ARRAY,PLANNED_DATE,DATE_ISSUE_DEATH_CERTIFICATE,DATE_TASK_SOLUTION_MIN,DATE_TASK_SOLUTION_MAX,REASON_ENOUGH

In [102]:
def merge_dicts(dicts):
    result_dict = {}
    for dictionary in dicts:
        if isinstance(dictionary, dict):  # Убедимся, что элемент является словарем
            for key, value in dictionary.items():
                result_dict[key] = value  # Обновление значения для ключа
    return result_dict
import pandas as pd


# Группировка по script_id и агрегация словарей в parameters
aggregated_df = df.groupby('script_id')['parameters'].agg(merge_dicts).reset_index()


aggregated_df

,script_id,parameters
0,17424b97-bf0a-45f9-9ce2-062eb39f3c37,"{'TYPE_EQUIPMENT': 'оборудованием', 'CATALOG':..."
1,7e3cfde7-53a7-40a9-b814-c373df9d8d04,"{'GET_CLIENT_TIMEZONE': '+03:00', 'EMPLOYEE_MR..."
2,a67dfe4b-a531-4366-9d7c-cfa348256b20,"{'COMMUNICATION_NUMBER': '000000344206174', 'C..."


In [103]:
# проверим сколько у нас всего script_id
df['script_id'].value_counts() 

script_id
7e3cfde7-53a7-40a9-b814-c373df9d8d04    501
a67dfe4b-a531-4366-9d7c-cfa348256b20    160
17424b97-bf0a-45f9-9ce2-062eb39f3c37     99
Name: count, dtype: int64

In [104]:
import pandas as pd
from pandas import json_normalize

def recursive_normalize(data, prefix=''):
    # Проверяем, является ли входящий элемент словарем
    if isinstance(data, dict):
        # Нормализация словаря на текущем уровне
        df = json_normalize(data)
    elif isinstance(data, list) and all(isinstance(x, dict) for x in data):
        # Если элемент - список словарей, нормализуем каждый словарь отдельно
        df = pd.concat([recursive_normalize(item, prefix) for item in data], ignore_index=True)
    else:
        # Если это не словарь и не список словарей, возвращаем пустой DataFrame
        return pd.DataFrame()

    # Обновляем имена колонок, добавляя префикс
    df.columns = [f'{prefix}{col}' if prefix else col for col in df.columns]
    return df

def expand_nested_columns(df, column):
    # Применяем рекурсивную функцию к каждой строке данных в заданной колонке
    expanded_frames = [recursive_normalize(row) for row in df[column]]
    expanded_df = pd.concat(expanded_frames, ignore_index=True)
    
    # Объединяем с исходным DataFrame по индексу
    final_df = df.drop(columns=[column]).join(expanded_df)
    return final_df


result_dff = expand_nested_columns(aggregated_df, 'parameters')

result_dff

,script_id,TYPE_EQUIPMENT,$$$taskId,$$$system,BUSINESS_KEY,NO_CACHE,HAS_IN_CATALOG,GET_CATALOG_RECORDS_QUANTITY,CLIENT_ID,ACCOUNT_NUMBER,HARDWARE_AVAILABILITY_AXIROS,CONNECTION_DIAGRAM,THERE_ROUTER,OPERATING_SYSTEM,PROBLEM_RESOLVED,REG_REASON,COMMUNICATION_THEME,COMMUNICATION_DETAIL,COMMUNICATION_RESULT,GUID_THEME,GUID_DETAIL,GUID_RESULT,COMMUNICATION_NUMBER,MARKETING_OFFERS,STATUS_MARKETING_OFFER,REPLACING_CABLE,THERE_IS_A_PHONE_NUMBER_FOR_THE_JOB,CLIENT_EQUIPMENT_BRAND,CLIENT_EQUIPMENT_ID,CLIENT_EQUIPMENT_MODEL,CLIENT_EQUIPMENT_HARDWARE_VERSION,LAST_EQUIPMENT_NETWORK_APPEARANCE_DATE_TIME,MRF,REGION_FILIAL,PORT_HSI_SERVICES,LAN_CONNECTION,OPERATING_SYSTEM_NEW,CLIENT_APPEAL,SSO_ID,COMMUNICATION_CHANNEL,COMMUNICATION_DIRECTION,SYSTEM_LOGIN,EMPLOYEE_TIMEZONE,SERVICE_ID,CRM_DEPARTMENT_INITIATOR,NEW_CLIENT,PROBLEM_NUMBER,TASK_NUMBER,PRODUCT_ID,CRM_LOGIN,COMMENT,HAS_AVAILABLE_OFFERS,CABLE_WIFI_CONNECTION,CONNECTION_METHOD_,RETURN_TIMEZONE,CONNECTION_METHOD,SELECTED_PRODUCT,CATALOG.catalogSlug,CATALOG_FILTER.filters,THEME_DETAIL_RESULT.theme,THEME_DETAIL_RESULT.theme-code,THEME_DETAIL_RESULT.theme-guid,THEME_DETAIL_RESULT.detailing,THEME_DETAIL_RESULT.detailing-code,THEME_DETAIL_RESULT.detailing-guid,THEME_DETAIL_RESULT.result,THEME_DETAIL_RESULT.result-code,THEME_DETAIL_RESULT.result-guid,GET_CLIENT_TIMEZONE,EMPLOYEE_MRF,CLIENT_TIMEZONE,FOR_LAST,FOR_LAST_UNIT,INTERACTION_TOPICS,HAS_COMMENT,GET_TASK_QUANTITY,TASK_QUANTITY_NEW,TASK_ARRAY,SEARCH_CASE_STATE,CASE_TYPES,CHECK_NSC,TESTER,GET_CLIENT_PROFILE_ID,GET_SEGMENT,GET_SUBSEGMENT,GET_CONVERGENT_ACCOUNT_FLAG,GET_CONVERGENT_ACCOUNT_NUMBER,PAYMENT_SCHEME,GET_ELK_ID,GET_ELK_LOGIN_BY_ACCOUNT_NUMBER,GET_ACCOUNT_STATUS,SERVICE_ID_2,ACCOUNT_NUMBER_1,SERVICE_ID_1,PRODUCT_ID_1,ADDENDUM_1,ADDENDUM_2,ADDENDUM_3,ADDENDUM_4,ADDENDUM_5,ROUND_UP,ADDITION,SELECTED_SERVICES_QUANTITY_,TASK_PRIORITY,TASK_FEEDBACK_TYPE,TASK_FEEDBACK_TIME_FROM,TASK_FEEDBACK_TIME_TO,CRM_DEPARTMENT,CLIENT_ID_AMDOCS,ORGANIZATION_ID,TASK_FEEDBACK_TYPE_VALUE,CALLER_ID,GET_EMAIL,GET_MOBILE_PHONE,GET_CONTACT_PHONE,TERMINATION_SERVICES,AMOUNT_RECALCULATION_MEETS_CRITERIA,TIMEZONE,COMMUNICATION_ARRAY,COMMUNICATION_QUANTITY_NEW,CATALOG_FIELD_VALUE_TEXT,CATALOG_FIELD_VALUE_DATE,CHECKING_POSSIBILITY_CHANGING_CONNECTION,COMPLETE_DIAGNOSIS,COMMUNICATION_ID,EMPLOYEE_SECOND_NAME,EMPLOYEE_FIRST_NAME,EMPLOYEE_EMAIL,CLIENT_TYPE,CLIENT_SECOND_NAME,CLIENT_FIRST_NAME,CLIENT_MIDDLE_NAME,CLIENT_MRF,SCRIPT_PLAYER_ID,HAS_ACTIVE_CALL,SKILL_GROUP_NAME,SEARCH_REASON,SYSTEM_SOURCE,REGIONAL_OFFICE_ADDRESS,CHOOSING_ANOTHER_REASON,LP_LINK,GET_CLIENT_TYPE,GET_ORGANIZATION_ID,GET_CLIENT_IS_TELE2,GET_CLIENT_IS_DZO,GET_DZO_NAME,CLIENT_DZO,SET_INTERACTION_SERVICE,SELECTED_SERVICES_QUANTITY,PERIOD_START_DATE_5,SERVICE_NAME_2,TASK_CREATE_DATE,VISIT_DATE,PROBLEM_DATE_BEGIN,CURRENT_SCRIPT_SUBOWNER_LOGIN,MAIN_SCRIPT_RUN_ID,PRODUCT_NAME_2,CURRENT_SCRIPT_NAME,TARGET_SCRIPT_NAME,SERVICE_LOGIN_1,PERIOD_END_DATE,PROBLEM_CLOSE_DATE,FLAT_NUMBER_2,SERVICE_LOGIN_2,PRODUCT_ID_2,NUMBER_TRANSFER_ DATE,SERVICE_END_DATE_3,PERIOD_START_DATE_4,SERVICE_START_DATE,HOUSE_ID_1,PERIOD_END_DATE_3,PERIOD_END_DATE_4,VISIT_TIME_FROM,PERIOD_START_DATE_3,INSTALLER_VISIT_TIME_TO,PERIOD_START_DATE_1,SERVICE_STATUS_2,ERROR_INTEGRATION_DATETIME,FLAT_NUMBER_1,LINE_NUMBER_1,PERIOD_END_DATE_1,SERVICE_START_DATE_3,SERVICE_END_DATE_4,MAIN_SCRIPT_OWNER_LOGIN,CURRENT_SCRIPT_VERSION,BIRTH_DATE,PERIOD_END_DATE_2,SERVICE_STATUS_1,ID_CARD_END_DATE,INSTALLER_VISIT_DATE,INSTALLER_VISIT_TIME_FROM,LINE_NUMBER_2,MAIN_SCRIPT_ID,PERIOD_START_DATE,CURRENT_SCRIPT_ID,SERVICE_NAME_1,SERVICE_END_DATE,SERVICE_START_DATE_4,VISIT_TIME_TO,TASK_CLOSE_DATE,SERVICE_END_DATE_2,SERVICE_END_DATE_5,ACCOUNT_NUMBER_2,PERIOD_START_DATE_2,DESIRABLE_TERMINATION_DATE,SELECTED_SERVICES_LIST,SERVICE_ADDRESS_1,SERVICE_START_DATE_5,SERVICE_END_DATE_1,SERVICE_START_DATE_2,SELECTED_PRODUCTS_LIST,SERVICE_START_DATE_1,PERIOD_END_DATE_5,HOUSE_ID_2,PRODUCT_NAME_1,ID_CARD_ISSUE_DATE,SERVICE_ADDRESS_2,DESIRED_TERMINATION_DATE,CURRENT_SCRIPT_RUN

In [106]:
def find_dict_columns(df):
    dict_columns = []
    for col in df.columns:
        # Проверяем, является ли хотя бы один элемент в колонке словарем
        if any(isinstance(cell, dict) for cell in df[col]):
            dict_columns.append(col)
    return dict_columns
def find_list_columns(df):
    list_columns = []
    for col in df.columns:
        # Проверяем, является ли хотя бы один элемент в колонке списком
        if any(isinstance(cell, list) for cell in df[col]):
            list_columns.append(col)
    return list_columns

def expand_dict_columns(df, dict_columns):
    for col in dict_columns:
        # Расширяем каждую колонку, содержащую словари
        expanded_data = df[col].apply(pd.Series)
        # Создаем новые имена колонок, добавляя префикс
        expanded_data.columns = [f'{col}.{subcol}' for subcol in expanded_data.columns]
        # Удаляем исходную колонку из DataFrame
        df = df.drop(col, axis=1)
        # Добавляем новые колонки в DataFrame
        df = pd.concat([df, expanded_data], axis=1)
    return df

In [107]:
# Смотрим сколько еще осталось списков 
dict_columns = find_list_columns(result_dff)
dict_columns

['CATALOG_FILTER.filters',
 'INTERACTION_TOPICS',
 'TASK_ARRAY',
 'CASE_TYPES',
 'COMMUNICATION_ARRAY',
 'GET_FILTERED_SERVICES_ARRAY',
 'PRODUCT_ARRAY',
 'STATUSES',
 'PRODUCT_IDS',
 'PROBLEM_ARRAY',
 'TASK_DESCRIPTION.conditions',
 'SERVICE_NAME.conditions',
 'SERVICE_STATUS.conditions',
 'PRODUCT_NAME.conditions',
 'SERVICE_TYPE.conditions',
 'PRICE_PLAN_NAME.conditions',
 'CONDITION_COMMENT.conditions']

In [108]:
def extract_first_list_element(df, list_columns):
    # Применяем изменения только к колонкам в list_columns
    for col in list_columns:
        df[col] = df[col].apply(lambda x: x[0] if isinstance(x, list) and x else None)
    return df
# Найдем колонки, содержащие списки
list_columns = find_list_columns(result_dff)

# Применяем функцию для извлечения первого элемента из списков в этих колонках
df = extract_first_list_element(result_dff, list_columns)

df

,script_id,TYPE_EQUIPMENT,$$$taskId,$$$system,BUSINESS_KEY,NO_CACHE,HAS_IN_CATALOG,GET_CATALOG_RECORDS_QUANTITY,CLIENT_ID,ACCOUNT_NUMBER,HARDWARE_AVAILABILITY_AXIROS,CONNECTION_DIAGRAM,THERE_ROUTER,OPERATING_SYSTEM,PROBLEM_RESOLVED,REG_REASON,COMMUNICATION_THEME,COMMUNICATION_DETAIL,COMMUNICATION_RESULT,GUID_THEME,GUID_DETAIL,GUID_RESULT,COMMUNICATION_NUMBER,MARKETING_OFFERS,STATUS_MARKETING_OFFER,REPLACING_CABLE,THERE_IS_A_PHONE_NUMBER_FOR_THE_JOB,CLIENT_EQUIPMENT_BRAND,CLIENT_EQUIPMENT_ID,CLIENT_EQUIPMENT_MODEL,CLIENT_EQUIPMENT_HARDWARE_VERSION,LAST_EQUIPMENT_NETWORK_APPEARANCE_DATE_TIME,MRF,REGION_FILIAL,PORT_HSI_SERVICES,LAN_CONNECTION,OPERATING_SYSTEM_NEW,CLIENT_APPEAL,SSO_ID,COMMUNICATION_CHANNEL,COMMUNICATION_DIRECTION,SYSTEM_LOGIN,EMPLOYEE_TIMEZONE,SERVICE_ID,CRM_DEPARTMENT_INITIATOR,NEW_CLIENT,PROBLEM_NUMBER,TASK_NUMBER,PRODUCT_ID,CRM_LOGIN,COMMENT,HAS_AVAILABLE_OFFERS,CABLE_WIFI_CONNECTION,CONNECTION_METHOD_,RETURN_TIMEZONE,CONNECTION_METHOD,SELECTED_PRODUCT,CATALOG.catalogSlug,CATALOG_FILTER.filters,THEME_DETAIL_RESULT.theme,THEME_DETAIL_RESULT.theme-code,THEME_DETAIL_RESULT.theme-guid,THEME_DETAIL_RESULT.detailing,THEME_DETAIL_RESULT.detailing-code,THEME_DETAIL_RESULT.detailing-guid,THEME_DETAIL_RESULT.result,THEME_DETAIL_RESULT.result-code,THEME_DETAIL_RESULT.result-guid,GET_CLIENT_TIMEZONE,EMPLOYEE_MRF,CLIENT_TIMEZONE,FOR_LAST,FOR_LAST_UNIT,INTERACTION_TOPICS,HAS_COMMENT,GET_TASK_QUANTITY,TASK_QUANTITY_NEW,TASK_ARRAY,SEARCH_CASE_STATE,CASE_TYPES,CHECK_NSC,TESTER,GET_CLIENT_PROFILE_ID,GET_SEGMENT,GET_SUBSEGMENT,GET_CONVERGENT_ACCOUNT_FLAG,GET_CONVERGENT_ACCOUNT_NUMBER,PAYMENT_SCHEME,GET_ELK_ID,GET_ELK_LOGIN_BY_ACCOUNT_NUMBER,GET_ACCOUNT_STATUS,SERVICE_ID_2,ACCOUNT_NUMBER_1,SERVICE_ID_1,PRODUCT_ID_1,ADDENDUM_1,ADDENDUM_2,ADDENDUM_3,ADDENDUM_4,ADDENDUM_5,ROUND_UP,ADDITION,SELECTED_SERVICES_QUANTITY_,TASK_PRIORITY,TASK_FEEDBACK_TYPE,TASK_FEEDBACK_TIME_FROM,TASK_FEEDBACK_TIME_TO,CRM_DEPARTMENT,CLIENT_ID_AMDOCS,ORGANIZATION_ID,TASK_FEEDBACK_TYPE_VALUE,CALLER_ID,GET_EMAIL,GET_MOBILE_PHONE,GET_CONTACT_PHONE,TERMINATION_SERVICES,AMOUNT_RECALCULATION_MEETS_CRITERIA,TIMEZONE,COMMUNICATION_ARRAY,COMMUNICATION_QUANTITY_NEW,CATALOG_FIELD_VALUE_TEXT,CATALOG_FIELD_VALUE_DATE,CHECKING_POSSIBILITY_CHANGING_CONNECTION,COMPLETE_DIAGNOSIS,COMMUNICATION_ID,EMPLOYEE_SECOND_NAME,EMPLOYEE_FIRST_NAME,EMPLOYEE_EMAIL,CLIENT_TYPE,CLIENT_SECOND_NAME,CLIENT_FIRST_NAME,CLIENT_MIDDLE_NAME,CLIENT_MRF,SCRIPT_PLAYER_ID,HAS_ACTIVE_CALL,SKILL_GROUP_NAME,SEARCH_REASON,SYSTEM_SOURCE,REGIONAL_OFFICE_ADDRESS,CHOOSING_ANOTHER_REASON,LP_LINK,GET_CLIENT_TYPE,GET_ORGANIZATION_ID,GET_CLIENT_IS_TELE2,GET_CLIENT_IS_DZO,GET_DZO_NAME,CLIENT_DZO,SET_INTERACTION_SERVICE,SELECTED_SERVICES_QUANTITY,PERIOD_START_DATE_5,SERVICE_NAME_2,TASK_CREATE_DATE,VISIT_DATE,PROBLEM_DATE_BEGIN,CURRENT_SCRIPT_SUBOWNER_LOGIN,MAIN_SCRIPT_RUN_ID,PRODUCT_NAME_2,CURRENT_SCRIPT_NAME,TARGET_SCRIPT_NAME,SERVICE_LOGIN_1,PERIOD_END_DATE,PROBLEM_CLOSE_DATE,FLAT_NUMBER_2,SERVICE_LOGIN_2,PRODUCT_ID_2,NUMBER_TRANSFER_ DATE,SERVICE_END_DATE_3,PERIOD_START_DATE_4,SERVICE_START_DATE,HOUSE_ID_1,PERIOD_END_DATE_3,PERIOD_END_DATE_4,VISIT_TIME_FROM,PERIOD_START_DATE_3,INSTALLER_VISIT_TIME_TO,PERIOD_START_DATE_1,SERVICE_STATUS_2,ERROR_INTEGRATION_DATETIME,FLAT_NUMBER_1,LINE_NUMBER_1,PERIOD_END_DATE_1,SERVICE_START_DATE_3,SERVICE_END_DATE_4,MAIN_SCRIPT_OWNER_LOGIN,CURRENT_SCRIPT_VERSION,BIRTH_DATE,PERIOD_END_DATE_2,SERVICE_STATUS_1,ID_CARD_END_DATE,INSTALLER_VISIT_DATE,INSTALLER_VISIT_TIME_FROM,LINE_NUMBER_2,MAIN_SCRIPT_ID,PERIOD_START_DATE,CURRENT_SCRIPT_ID,SERVICE_NAME_1,SERVICE_END_DATE,SERVICE_START_DATE_4,VISIT_TIME_TO,TASK_CLOSE_DATE,SERVICE_END_DATE_2,SERVICE_END_DATE_5,ACCOUNT_NUMBER_2,PERIOD_START_DATE_2,DESIRABLE_TERMINATION_DATE,SELECTED_SERVICES_LIST,SERVICE_ADDRESS_1,SERVICE_START_DATE_5,SERVICE_END_DATE_1,SERVICE_START_DATE_2,SELECTED_PRODUCTS_LIST,SERVICE_START_DATE_1,PERIOD_END_DATE_5,HOUSE_ID_2,PRODUCT_NAME_1,ID_CARD_ISSUE_DATE,SERVICE_ADDRESS_2,DESIRED_TERMINATION_DATE,CURRENT_SCRIPT_RUN

# Рекурсивно проходимся по данным: убираем словари и списки

In [109]:
import pandas as pd

def find_dict_columns(df):
    dict_columns = []
    for col in df.columns:
        if any(isinstance(cell, dict) for cell in df[col]):
            dict_columns.append(col)
    return dict_columns

def find_list_columns(df):
    list_columns = []
    for col in df.columns:
        if any(isinstance(cell, list) for cell in df[col]):
            list_columns.append(col)
    return list_columns

def expand_dict_columns(df, dict_columns):
    for col in dict_columns:
        expanded_data = df[col].apply(pd.Series)
        expanded_data.columns = [f'{col}.{subcol}' for subcol in expanded_data.columns]
        df = df.drop(col, axis=1)
        df = pd.concat([df, expanded_data], axis=1)
    return df

def extract_first_list_item(df, list_columns):
    updated_list_columns = []
    original_data = df[list_columns].copy()  # Сохраняем оригинальные данные для сравнения

    for col in list_columns:
        df[col] = df[col].apply(
            lambda x: x[0] if isinstance(x, list) and len(x) > 0 and not isinstance(x[0], list) and (not isinstance(x[0], dict) or (isinstance(x[0], dict) and x[0])) else x
        )
        if not df[col].equals(original_data[col]) and df[col].notna().any():
            updated_list_columns.append(col)
        print(f"Column {col} after processing lists, skipping empty dicts:\n{df[col].head()}")

    return df, updated_list_columns, original_data

def process_dataframe(df, depth=0, max_depth=None):
    indent = "  " * depth
    print(f"{indent}Call to process_dataframe at depth {depth}")

    if max_depth is not None and depth >= max_depth:
        print(f"{indent}Reached max recursion depth of {max_depth}. Exiting recursion.")
        return df

    list_columns = find_list_columns(df)
    if list_columns:
        print(f"{indent}Found lists in columns: {list_columns}")
        df, updated_list_columns, original_data = extract_first_list_item(df, list_columns)
        if updated_list_columns:
            print(f"{indent}Processed lists, recursing...")
            df = process_dataframe(df, depth + 1, max_depth)
        # Восстанавливаем колонки с неизмененными данными
        for col in list_columns:
            if col not in updated_list_columns:
                df[col] = original_data[col]
        return df

    dict_columns = find_dict_columns(df)
    if dict_columns:
        print(f"{indent}Found dictionaries in columns: {dict_columns}")
        df = expand_dict_columns(df, dict_columns)
        print(f"{indent}Expanded dictionaries, recursing...")
        df = process_dataframe(df, depth + 1, max_depth)
        return df

    print(f"{indent}No more lists or dictionaries to process. Exiting recursion.")
    return df

df = result_dff

# Обработка DataFrame
result_ddf = process_dataframe(df)
print("Final result structure:")
print(result_ddf.dtypes)

Call to process_dataframe at depth 0
Found dictionaries in columns: ['CATALOG_FILTER.filters', 'INTERACTION_TOPICS', 'CASE_TYPES', 'GET_FILTERED_SERVICES_ARRAY', 'PRODUCT_ARRAY', 'TASK_DESCRIPTION.conditions', 'SERVICE_NAME.conditions', 'SERVICE_STATUS.conditions', 'PRODUCT_NAME.conditions', 'SERVICE_TYPE.conditions', 'PRICE_PLAN_NAME.conditions', 'CONDITION_COMMENT.conditions']
Expanded dictionaries, recursing...
  Call to process_dataframe at depth 1
  Found lists in columns: ['CATALOG_FILTER.filters.templates']
Column CATALOG_FILTER.filters.templates after processing lists, skipping empty dicts:
0    {'operator': 'eq', 'template': 'Волга', 'kind'...
1    {'operator': 'include', 'template': 'main_scri...
2    {'operator': 'include', 'template': 'task_feed...
Name: CATALOG_FILTER.filters.templates, dtype: object
  Processed lists, recursing...
    Call to process_dataframe at depth 2
    Found dictionaries in columns: ['CATALOG_FILTER.filters.templates', 'INTERACTION_TOPICS.theme', 'I

In [110]:
result_ddf

,script_id,TYPE_EQUIPMENT,$$$taskId,$$$system,BUSINESS_KEY,NO_CACHE,HAS_IN_CATALOG,GET_CATALOG_RECORDS_QUANTITY,CLIENT_ID,ACCOUNT_NUMBER,HARDWARE_AVAILABILITY_AXIROS,CONNECTION_DIAGRAM,THERE_ROUTER,OPERATING_SYSTEM,PROBLEM_RESOLVED,REG_REASON,COMMUNICATION_THEME,COMMUNICATION_DETAIL,COMMUNICATION_RESULT,GUID_THEME,GUID_DETAIL,GUID_RESULT,COMMUNICATION_NUMBER,MARKETING_OFFERS,STATUS_MARKETING_OFFER,REPLACING_CABLE,THERE_IS_A_PHONE_NUMBER_FOR_THE_JOB,CLIENT_EQUIPMENT_BRAND,CLIENT_EQUIPMENT_ID,CLIENT_EQUIPMENT_MODEL,CLIENT_EQUIPMENT_HARDWARE_VERSION,LAST_EQUIPMENT_NETWORK_APPEARANCE_DATE_TIME,MRF,REGION_FILIAL,PORT_HSI_SERVICES,LAN_CONNECTION,OPERATING_SYSTEM_NEW,CLIENT_APPEAL,SSO_ID,COMMUNICATION_CHANNEL,COMMUNICATION_DIRECTION,SYSTEM_LOGIN,EMPLOYEE_TIMEZONE,SERVICE_ID,CRM_DEPARTMENT_INITIATOR,NEW_CLIENT,PROBLEM_NUMBER,TASK_NUMBER,PRODUCT_ID,CRM_LOGIN,COMMENT,HAS_AVAILABLE_OFFERS,CABLE_WIFI_CONNECTION,CONNECTION_METHOD_,RETURN_TIMEZONE,CONNECTION_METHOD,SELECTED_PRODUCT,CATALOG.catalogSlug,THEME_DETAIL_RESULT.theme,THEME_DETAIL_RESULT.theme-code,THEME_DETAIL_RESULT.theme-guid,THEME_DETAIL_RESULT.detailing,THEME_DETAIL_RESULT.detailing-code,THEME_DETAIL_RESULT.detailing-guid,THEME_DETAIL_RESULT.result,THEME_DETAIL_RESULT.result-code,THEME_DETAIL_RESULT.result-guid,GET_CLIENT_TIMEZONE,EMPLOYEE_MRF,CLIENT_TIMEZONE,FOR_LAST,FOR_LAST_UNIT,HAS_COMMENT,GET_TASK_QUANTITY,TASK_QUANTITY_NEW,TASK_ARRAY,SEARCH_CASE_STATE,CHECK_NSC,TESTER,GET_CLIENT_PROFILE_ID,GET_SEGMENT,GET_SUBSEGMENT,GET_CONVERGENT_ACCOUNT_FLAG,GET_CONVERGENT_ACCOUNT_NUMBER,PAYMENT_SCHEME,GET_ELK_ID,GET_ELK_LOGIN_BY_ACCOUNT_NUMBER,GET_ACCOUNT_STATUS,SERVICE_ID_2,ACCOUNT_NUMBER_1,SERVICE_ID_1,PRODUCT_ID_1,ADDENDUM_1,ADDENDUM_2,ADDENDUM_3,ADDENDUM_4,ADDENDUM_5,ROUND_UP,ADDITION,SELECTED_SERVICES_QUANTITY_,TASK_PRIORITY,TASK_FEEDBACK_TYPE,TASK_FEEDBACK_TIME_FROM,TASK_FEEDBACK_TIME_TO,CRM_DEPARTMENT,CLIENT_ID_AMDOCS,ORGANIZATION_ID,TASK_FEEDBACK_TYPE_VALUE,CALLER_ID,GET_EMAIL,GET_MOBILE_PHONE,GET_CONTACT_PHONE,TERMINATION_SERVICES,AMOUNT_RECALCULATION_MEETS_CRITERIA,TIMEZONE,COMMUNICATION_ARRAY,COMMUNICATION_QUANTITY_NEW,CATALOG_FIELD_VALUE_TEXT,CATALOG_FIELD_VALUE_DATE,CHECKING_POSSIBILITY_CHANGING_CONNECTION,COMPLETE_DIAGNOSIS,COMMUNICATION_ID,EMPLOYEE_SECOND_NAME,EMPLOYEE_FIRST_NAME,EMPLOYEE_EMAIL,CLIENT_TYPE,CLIENT_SECOND_NAME,CLIENT_FIRST_NAME,CLIENT_MIDDLE_NAME,CLIENT_MRF,SCRIPT_PLAYER_ID,HAS_ACTIVE_CALL,SKILL_GROUP_NAME,SEARCH_REASON,SYSTEM_SOURCE,REGIONAL_OFFICE_ADDRESS,CHOOSING_ANOTHER_REASON,LP_LINK,GET_CLIENT_TYPE,GET_ORGANIZATION_ID,GET_CLIENT_IS_TELE2,GET_CLIENT_IS_DZO,GET_DZO_NAME,CLIENT_DZO,SET_INTERACTION_SERVICE,SELECTED_SERVICES_QUANTITY,PERIOD_START_DATE_5,SERVICE_NAME_2,TASK_CREATE_DATE,VISIT_DATE,PROBLEM_DATE_BEGIN,CURRENT_SCRIPT_SUBOWNER_LOGIN,MAIN_SCRIPT_RUN_ID,PRODUCT_NAME_2,CURRENT_SCRIPT_NAME,TARGET_SCRIPT_NAME,SERVICE_LOGIN_1,PERIOD_END_DATE,PROBLEM_CLOSE_DATE,FLAT_NUMBER_2,SERVICE_LOGIN_2,PRODUCT_ID_2,NUMBER_TRANSFER_ DATE,SERVICE_END_DATE_3,PERIOD_START_DATE_4,SERVICE_START_DATE,HOUSE_ID_1,PERIOD_END_DATE_3,PERIOD_END_DATE_4,VISIT_TIME_FROM,PERIOD_START_DATE_3,INSTALLER_VISIT_TIME_TO,PERIOD_START_DATE_1,SERVICE_STATUS_2,ERROR_INTEGRATION_DATETIME,FLAT_NUMBER_1,LINE_NUMBER_1,PERIOD_END_DATE_1,SERVICE_START_DATE_3,SERVICE_END_DATE_4,MAIN_SCRIPT_OWNER_LOGIN,CURRENT_SCRIPT_VERSION,BIRTH_DATE,PERIOD_END_DATE_2,SERVICE_STATUS_1,ID_CARD_END_DATE,INSTALLER_VISIT_DATE,INSTALLER_VISIT_TIME_FROM,LINE_NUMBER_2,MAIN_SCRIPT_ID,PERIOD_START_DATE,CURRENT_SCRIPT_ID,SERVICE_NAME_1,SERVICE_END_DATE,SERVICE_START_DATE_4,VISIT_TIME_TO,TASK_CLOSE_DATE,SERVICE_END_DATE_2,SERVICE_END_DATE_5,ACCOUNT_NUMBER_2,PERIOD_START_DATE_2,DESIRABLE_TERMINATION_DATE,SELECTED_SERVICES_LIST,SERVICE_ADDRESS_1,SERVICE_START_DATE_5,SERVICE_END_DATE_1,SERVICE_START_DATE_2,SELECTED_PRODUCTS_LIST,SERVICE_START_DATE_1,PERIOD_END_DATE_5,HOUSE_ID_2,PRODUCT_NAME_1,ID_CARD_ISSUE_DATE,SERVICE_ADDRESS_2,DESIRED_TERMINATION_DATE,CURRENT_SCRIPT_RUN_ID,MAIN_SCRIPT_SUBOWNER_LOGIN,CURRENT_SCRIPT_OWNER_L

In [111]:
result_ddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Columns: 491 entries, script_id to CASE_TYPES.type1.guid
dtypes: bool(3), float64(21), int64(1), object(466)
memory usage: 11.6+ KB


In [15]:
#result_ddf.to_csv("final_last_version.csv")

In [112]:
# Функция для очистки имен столбцов. Убираем все $ из названия столбцов
import re
result_data = result_ddf
def clean_column_names(columns):
    cleaned_columns = {}
    for col in columns:
        cleaned_col = re.sub(r'[^a-zA-Z0-9_]', '', col)
        cleaned_columns[col] = cleaned_col
    return cleaned_columns

# Применение функции для очистки имен столбцов в DataFrame
cleaned_column_names = clean_column_names(result_data.columns)
result_data.rename(columns=cleaned_column_names, inplace=True)

# Загружаем данные в кликхауз

In [114]:
result_data

,script_id,TYPE_EQUIPMENT,taskId,system,BUSINESS_KEY,NO_CACHE,HAS_IN_CATALOG,GET_CATALOG_RECORDS_QUANTITY,CLIENT_ID,ACCOUNT_NUMBER,HARDWARE_AVAILABILITY_AXIROS,CONNECTION_DIAGRAM,THERE_ROUTER,OPERATING_SYSTEM,PROBLEM_RESOLVED,REG_REASON,COMMUNICATION_THEME,COMMUNICATION_DETAIL,COMMUNICATION_RESULT,GUID_THEME,GUID_DETAIL,GUID_RESULT,COMMUNICATION_NUMBER,MARKETING_OFFERS,STATUS_MARKETING_OFFER,REPLACING_CABLE,THERE_IS_A_PHONE_NUMBER_FOR_THE_JOB,CLIENT_EQUIPMENT_BRAND,CLIENT_EQUIPMENT_ID,CLIENT_EQUIPMENT_MODEL,CLIENT_EQUIPMENT_HARDWARE_VERSION,LAST_EQUIPMENT_NETWORK_APPEARANCE_DATE_TIME,MRF,REGION_FILIAL,PORT_HSI_SERVICES,LAN_CONNECTION,OPERATING_SYSTEM_NEW,CLIENT_APPEAL,SSO_ID,COMMUNICATION_CHANNEL,COMMUNICATION_DIRECTION,SYSTEM_LOGIN,EMPLOYEE_TIMEZONE,SERVICE_ID,CRM_DEPARTMENT_INITIATOR,NEW_CLIENT,PROBLEM_NUMBER,TASK_NUMBER,PRODUCT_ID,CRM_LOGIN,COMMENT,HAS_AVAILABLE_OFFERS,CABLE_WIFI_CONNECTION,CONNECTION_METHOD_,RETURN_TIMEZONE,CONNECTION_METHOD,SELECTED_PRODUCT,CATALOGcatalogSlug,THEME_DETAIL_RESULTtheme,THEME_DETAIL_RESULTthemecode,THEME_DETAIL_RESULTthemeguid,THEME_DETAIL_RESULTdetailing,THEME_DETAIL_RESULTdetailingcode,THEME_DETAIL_RESULTdetailingguid,THEME_DETAIL_RESULTresult,THEME_DETAIL_RESULTresultcode,THEME_DETAIL_RESULTresultguid,GET_CLIENT_TIMEZONE,EMPLOYEE_MRF,CLIENT_TIMEZONE,FOR_LAST,FOR_LAST_UNIT,HAS_COMMENT,GET_TASK_QUANTITY,TASK_QUANTITY_NEW,TASK_ARRAY,SEARCH_CASE_STATE,CHECK_NSC,TESTER,GET_CLIENT_PROFILE_ID,GET_SEGMENT,GET_SUBSEGMENT,GET_CONVERGENT_ACCOUNT_FLAG,GET_CONVERGENT_ACCOUNT_NUMBER,PAYMENT_SCHEME,GET_ELK_ID,GET_ELK_LOGIN_BY_ACCOUNT_NUMBER,GET_ACCOUNT_STATUS,SERVICE_ID_2,ACCOUNT_NUMBER_1,SERVICE_ID_1,PRODUCT_ID_1,ADDENDUM_1,ADDENDUM_2,ADDENDUM_3,ADDENDUM_4,ADDENDUM_5,ROUND_UP,ADDITION,SELECTED_SERVICES_QUANTITY_,TASK_PRIORITY,TASK_FEEDBACK_TYPE,TASK_FEEDBACK_TIME_FROM,TASK_FEEDBACK_TIME_TO,CRM_DEPARTMENT,CLIENT_ID_AMDOCS,ORGANIZATION_ID,TASK_FEEDBACK_TYPE_VALUE,CALLER_ID,GET_EMAIL,GET_MOBILE_PHONE,GET_CONTACT_PHONE,TERMINATION_SERVICES,AMOUNT_RECALCULATION_MEETS_CRITERIA,TIMEZONE,COMMUNICATION_ARRAY,COMMUNICATION_QUANTITY_NEW,CATALOG_FIELD_VALUE_TEXT,CATALOG_FIELD_VALUE_DATE,CHECKING_POSSIBILITY_CHANGING_CONNECTION,COMPLETE_DIAGNOSIS,COMMUNICATION_ID,EMPLOYEE_SECOND_NAME,EMPLOYEE_FIRST_NAME,EMPLOYEE_EMAIL,CLIENT_TYPE,CLIENT_SECOND_NAME,CLIENT_FIRST_NAME,CLIENT_MIDDLE_NAME,CLIENT_MRF,SCRIPT_PLAYER_ID,HAS_ACTIVE_CALL,SKILL_GROUP_NAME,SEARCH_REASON,SYSTEM_SOURCE,REGIONAL_OFFICE_ADDRESS,CHOOSING_ANOTHER_REASON,LP_LINK,GET_CLIENT_TYPE,GET_ORGANIZATION_ID,GET_CLIENT_IS_TELE2,GET_CLIENT_IS_DZO,GET_DZO_NAME,CLIENT_DZO,SET_INTERACTION_SERVICE,SELECTED_SERVICES_QUANTITY,PERIOD_START_DATE_5,SERVICE_NAME_2,TASK_CREATE_DATE,VISIT_DATE,PROBLEM_DATE_BEGIN,CURRENT_SCRIPT_SUBOWNER_LOGIN,MAIN_SCRIPT_RUN_ID,PRODUCT_NAME_2,CURRENT_SCRIPT_NAME,TARGET_SCRIPT_NAME,SERVICE_LOGIN_1,PERIOD_END_DATE,PROBLEM_CLOSE_DATE,FLAT_NUMBER_2,SERVICE_LOGIN_2,PRODUCT_ID_2,NUMBER_TRANSFER_DATE,SERVICE_END_DATE_3,PERIOD_START_DATE_4,SERVICE_START_DATE,HOUSE_ID_1,PERIOD_END_DATE_3,PERIOD_END_DATE_4,VISIT_TIME_FROM,PERIOD_START_DATE_3,INSTALLER_VISIT_TIME_TO,PERIOD_START_DATE_1,SERVICE_STATUS_2,ERROR_INTEGRATION_DATETIME,FLAT_NUMBER_1,LINE_NUMBER_1,PERIOD_END_DATE_1,SERVICE_START_DATE_3,SERVICE_END_DATE_4,MAIN_SCRIPT_OWNER_LOGIN,CURRENT_SCRIPT_VERSION,BIRTH_DATE,PERIOD_END_DATE_2,SERVICE_STATUS_1,ID_CARD_END_DATE,INSTALLER_VISIT_DATE,INSTALLER_VISIT_TIME_FROM,LINE_NUMBER_2,MAIN_SCRIPT_ID,PERIOD_START_DATE,CURRENT_SCRIPT_ID,SERVICE_NAME_1,SERVICE_END_DATE,SERVICE_START_DATE_4,VISIT_TIME_TO,TASK_CLOSE_DATE,SERVICE_END_DATE_2,SERVICE_END_DATE_5,ACCOUNT_NUMBER_2,PERIOD_START_DATE_2,DESIRABLE_TERMINATION_DATE,SELECTED_SERVICES_LIST,SERVICE_ADDRESS_1,SERVICE_START_DATE_5,SERVICE_END_DATE_1,SERVICE_START_DATE_2,SELECTED_PRODUCTS_LIST,SERVICE_START_DATE_1,PERIOD_END_DATE_5,HOUSE_ID_2,PRODUCT_NAME_1,ID_CARD_ISSUE_DATE,SERVICE_ADDRESS_2,DESIRED_TERMINATION_DATE,CURRENT_SCRIPT_RUN_ID,MAIN_SCRIPT_SUBOWNER_LOGIN,CURRENT_SCRIPT_OWNER_LOGIN,MAIN_SCRIPT_VERSIO

In [115]:
final_dataset = result_data.copy() # делаю копию датафрейма

In [116]:
object_columns = final_dataset.select_dtypes(include=['object']).columns.tolist()

# Преобразование колонок типа object в строки
for col in object_columns:
    final_dataset[col] = final_dataset[col].astype(str)

In [117]:
from clickhouse_driver import Client
import pandas as pd

def determine_clickhouse_type(series):
    if pd.api.types.is_integer_dtype(series):
        return 'Int64'
    elif pd.api.types.is_float_dtype(series):
        return 'Float64'
    elif pd.api.types.is_datetime64_dtype(series):
        return 'Date'
    elif pd.api.types.is_bool_dtype(series):
        return 'UInt8'
    elif pd.api.types.is_object_dtype(series):
        if series.apply(lambda x: isinstance(x, str)).all():
            return 'String'
        else:
            return 'String'
    else:
        return 'String'

def create_table_query(table_name, columns_types, primary_key):
    columns_definition = ', '.join([f'{col} {typ}' for col, typ in columns_types.items()])
    query = f'CREATE TABLE IF NOT EXISTS db_test.{table_name} ({columns_definition}) ENGINE = MergeTree() ORDER BY ({primary_key});'
    return query

# Получаем типы данных для каждого столбца
clickhouse_types = {col: determine_clickhouse_type(final_dataset[col]) for col in final_dataset.columns}

# Создаем запрос на создание таблицы, указывая первый столбец в качестве первичного ключа
table_name = 'dataset_new'
primary_key = final_dataset.columns[0]  # сделаем script_id первичным ключем
create_query = create_table_query(table_name, clickhouse_types, primary_key)
print(create_query)

# Подключаемся к ClickHouse и выполняем запрос
client = Client(host='localhost', port=9000, user='default', password='pivanet')
client.execute(create_query)

print(f"Table {table_name} created successfully")

CREATE TABLE IF NOT EXISTS db_test.dataset_new (script_id String, TYPE_EQUIPMENT String, taskId String, system String, BUSINESS_KEY String, NO_CACHE UInt8, HAS_IN_CATALOG UInt8, GET_CATALOG_RECORDS_QUANTITY Int64, CLIENT_ID String, ACCOUNT_NUMBER String, HARDWARE_AVAILABILITY_AXIROS String, CONNECTION_DIAGRAM String, THERE_ROUTER String, OPERATING_SYSTEM String, PROBLEM_RESOLVED String, REG_REASON String, COMMUNICATION_THEME String, COMMUNICATION_DETAIL String, COMMUNICATION_RESULT String, GUID_THEME String, GUID_DETAIL String, GUID_RESULT String, COMMUNICATION_NUMBER String, MARKETING_OFFERS String, STATUS_MARKETING_OFFER String, REPLACING_CABLE String, THERE_IS_A_PHONE_NUMBER_FOR_THE_JOB String, CLIENT_EQUIPMENT_BRAND String, CLIENT_EQUIPMENT_ID String, CLIENT_EQUIPMENT_MODEL String, CLIENT_EQUIPMENT_HARDWARE_VERSION String, LAST_EQUIPMENT_NETWORK_APPEARANCE_DATE_TIME String, MRF String, REGION_FILIAL String, PORT_HSI_SERVICES String, LAN_CONNECTION String, OPERATING_SYSTEM_NEW Strin

In [118]:
import pandas as pd
import re
from clickhouse_driver import Client

# Подготовка данных для вставки
data_tuples = [tuple(x) for x in final_dataset.to_numpy()]
columns = ', '.join(final_dataset.columns)

# Запрос для вставки данных
insert_query = f'INSERT INTO db_test.dataset_new ({columns}) VALUES'

try:
    client.execute(insert_query, data_tuples)
    print(f"Data inserted into new_table successfully")
except Exception as e:
    print("Ошибка при вставке данных:", e)

Data inserted into new_table successfully
